In [3]:
import os
from dotenv import load_dotenv

from agents import Agent, WebSearchTool

In [ ]:
from dis import Instruction

from agents import ModelSettings


load_dotenv(override=True)

INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model='gpt-4.1-nano',
    model_settings=ModelSettings(tool_choice="required")
)